# Deep Learning: Convolutional Neural Network

### Importing the necessary Libraries

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [4]:
import keras.models as km

### Let's talk about the libraries and why do we need them

- Sequential library basically means that we are going to create linear stack of layers where the output of neurons of one layer will serve as the input to the next layer and in this way we'll get something like tree with roots coming out and getting more complex, more info [keras](https://keras.io/getting-started/sequential-model-guide/).
- Conv2D basically tries to fetch the important information from the image like curve, smoothness, or a particular patterns
- MaxPooling2D work is to reduce the size of the matrix without compromising with the information that it contains
- Flatten work is to convert the madrix to the one dimensional vector so that its each element could then be used as an input to the artificial neural network
- Dense is used to add more layer to the ann efficiently i suppose

### Initialising our classifier and telling that it has to be Sequential

In [5]:
classifier = Sequential()

### Adding the Convolution

In [8]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

#### Let's break the above code and understand it 
- 32 represents the number of features detector that we want
- (3, 3) represent thr strides size
- input_shape = (64, 64, 3), here 64,64 represent the size of the image and 3 represent that it is a color image
- activation = 'relu' actually we require a threshold to set, for example say we if some feature give us a value greater than the threshold then it fires up the neuron and if it is less then it does nothing, and this activation is calculated through 'relu', more on [StackOverflow](https://stackoverflow.com/questions/27319931/relu-and-dropout-in-cnn)

### Adding a Max Pooling layer

In [9]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

#### Max Pooling as mentioned above is used to decrease the dimension of the matrix so that we only pass the relevant information and not garbage
- pool_size: factors by which to downscale (vertical, horizontal). (2, 2) will halve the input in both spatial dimension. If only one integer is specified, the same window length will be used for both dimensions.


### We'll be adding another layer of convolution and Max Pooling so to further minimize the matrix and extract useful information out of it

In [12]:
classifier.add(Conv2D(32, (3, 3), activation='relu')) 
classifier.add(MaxPooling2D(pool_size = (2, 2)))

### Flattening the Matrix

In [13]:
classifier.add(Flatten())

#### Flatten()
- Flatten will convert the matrix into a column vector which then can be feed to the ann(Artificial Neural Network)

## Full Connection - Artificial Neural network

#### From here we have a single column vector that contains all the necessary information that we need to build a model
- we'll be building layers of neurons that will try to figure out the information from these datas that we figured out during convolution
- we'll predict the output and compare it to the truth value and we calculate the error
- we'll try to minimize the error through backpropagation in which we actually try to modify weights
- to get the most efficient weight we use stochastic gradient descent method, this method is like finding the minimum of the cost function or error function.
- we'll also use dropout, in which some neurons are randomly shut down, this helps us in reducing over fitting
- we'll run out the model again and again, epochs and will try to get the best model and will save it.
- saving the model help us from running the model through all the process again and again, because these computations are actually time consuming


In [15]:
classifier.add(Dense(units = 128, activation = 'relu'))

#### Dense:
- dense function actually helps us in creating the hidden layers efficiently
- units means the number of neurons in the second layer, here it is 128
- activation again tells us that when a neuron will fire up or lights up depending upon how much sure it is, and again we have choose 'relu'

In [17]:
classifier.add(Dense(units = 1, activation = 'sigmoid'))

#### Last neuron that will decide whether we classified dog or cat

### Compiling the CNN

In [18]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#### compile( )
- adam is actually like stochastic gradient descent, [more](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)
- loss here attributes about the loss function or cost function that we want to minimize using stochastic gradient descent
- metrics, want the attribute, what we want to increase, here we want our model to increase in accuracy

### Fitting the CNN with the data

- First we need to have the sufficient data so that our model can learn properly, say we have less data then we can use keras ImageDataGenerator, what it does is it will alter the current image, like zoom, crop, tilt, so that we will have varieties of the single photo and it will enrich our data
- we need to have the data classified for the training purposes

In [19]:
from keras.preprocessing.image import ImageDataGenerator

/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [21]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

#### ImageDataGenerator( )
- we want all the pixel value to range from 0 to 1 i.e why we rescale it using rescale=1./255
- shear_range, zoom_range, horizontal_flip these are the attributes that we can control to affect the images, so increasing zoom will result in a different photo than we had.

In [23]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


#### .flow_from_directory( )
- first parameter is to give the path of the folder where our images are stored
- second parameter resize the images into 64 64 size
- batch size actually means how much images it will consider in one batch and work on it
- binary means that we either will have 0 or 1 result, i.e cat or dog

### Fitting the data to the classifier, to learn

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch=8000,
                         epochs=30,
                         validation_data=test_set,
                         validation_steps=2000)

#### .fit_generator( )
- first parameter takes the set to which we want to train the data
- second parameter takes the steps_per_epochs, i.e in one epoch on how much data we'll work on as there is 8000 images in training set it is set to 8000
- third parameter takes how many epochs there will be
- fourth parameter will take the set from where it will validate its prediction, i.e. how accurate its prediction is
- fifth parameter takes how many images to check on, as we have 2000 images in the test set it is set to 2000

### Save the classifier 

In [ ]:
km.save_model(classifier,filepath = 'highly_trained.h5')

### Making a Single Prediction

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('dataset/dog_or_cat.jpeg',
                            target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0) # o means that we want to add the dimension in first place

result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'dog'
elif result[0][0] == 0:
    prediction = 'cat'

#### Let's break down the code
- first we are importing the essential library so that we can easily work on the image and apply certain changes like the size and change its dimension
- then we load the image using image.load_img(filepath, target_size =(64,64))
- then we connvert the image to array
- then we expand the dimension of the array as it required by the classifier
- then we use predict function of classifier on the image